In [4]:
import traitlets
import os
from jetbot import Camera, bgr8_to_jpeg
import ipywidgets.widgets as widgets
from IPython.display import display
from uuid import uuid1
import json
import glob

In [5]:
camera = Camera.instance(width=224, height=224)

image_widget = widgets.Image(format='jpeg', width=300, height=300)

traitlets.dlink((camera, 'value'), (image_widget, 'value'), transform=bgr8_to_jpeg)

display(image_widget)

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x02\x01\x0…

In [17]:
controller = widgets.Controller(index=1)

display(controller)

Controller(index=1)

# Collect steering data

In [18]:
DATASET_DIR = 'dataset_all'
#DATASET_DIR = 'dataset_left'
#DATASET_DIR = 'dataset_forward'
#DATASET_DIR = 'dataset_right'

LEFT_DIR = os.path.join(DATASET_DIR, 'left')
FORWARD_DIR = os.path.join(DATASET_DIR, 'forward')
RIGHT_DIR = os.path.join(DATASET_DIR, 'right')

# we have this "try/except" statement because these next functions can throw an error if the directories exist already
try:
    os.makedirs(LEFT_DIR)
    os.makedirs(FORWARD_DIR)
    os.makedirs(RIGHT_DIR)
except FileExistsError:
    print('Directories not created becasue they already exist')

    

for b in controller.buttons:
    b.unobserve_all()
for a in controller.axes:
    a.unobserve_all()
    
layout_btn = widgets.Layout(width='80px', height='80px', align_self='center')
# get controller buttons / axes
left_button = widgets.Button(description='left', layout=layout_btn)
def click_left(c):
    if c['new']:
        left_button.click()
controller.buttons[14].observe(click_left, names='value')

forward_button = widgets.Button(description='forward', layout=layout_btn)
def click_forward(c):
    if c['new']:
        forward_button.click()
controller.buttons[12].observe(click_forward, names='value')

right_button = widgets.Button(description='right', layout=layout_btn)
def click_right(c):
    if c['new']:
        right_button.click()
controller.buttons[15].observe(click_right, names='value')


# create widgets that will display the annotation to be saved
snapshot_widget = widgets.Image(format='jpeg', width=300, height=300)

def get_left_count():
    return len(glob.glob(os.path.join(LEFT_DIR, '*.jpg')))

def get_forward_count():
    return len(glob.glob(os.path.join(FORWARD_DIR, '*.jpg')))
               
def get_right_count():
    return len(glob.glob(os.path.join(RIGHT_DIR, '*.jpg')))

layout_cnt = widgets.Layout(width='80px', align_self='center')
left_count_widget = widgets.IntText(value=get_left_count(), layout=layout_cnt)
forward_count_widget = widgets.IntText(value=get_forward_count(), layout=layout_cnt, button_style='success')
right_count_widget = widgets.IntText(value=get_right_count(), layout=layout_cnt)

def save_left(c):
    save(LEFT_DIR)

def save_right(c):
    save(RIGHT_DIR)

def save_forward(c):
    save(FORWARD_DIR)
    
def save(directory):
    uuid = str(uuid1())
    image_name = uuid + '.jpg'
    image_path = os.path.join(directory, image_name)

    snapshot_widget.value = image_widget.value
    
    with open(image_path, 'wb') as f:
        f.write(snapshot_widget.value)
               
    left_count_widget.value = get_left_count()
    right_count_widget.value = get_right_count()
    forward_count_widget.value = get_forward_count()
        

left_button.on_click(save_left)
forward_button.on_click(save_forward)
right_button.on_click(save_right)


display(
  widgets.HBox([image_widget, snapshot_widget]),
  widgets.HBox([left_count_widget, forward_count_widget, right_count_widget]),
  widgets.HBox([left_button, forward_button, right_button])
)

Directories not created becasue they already exist


In [63]:
!zip -r -q lego_city_{DATASET_DIR}_{uuid1()}.zip {DATASET_DIR}

# Label the 7 block configurations

In [58]:
import copy

In [64]:
THUMBNAIL_DIR = 'thumbnails'
BLOCKS_DATASET_DIR = 'dataset_blocks'

class_names = [
    'cross',
    'left',
    'right',
    'straight',
    't_left',
    't_right',
    't_straight'
]


def get_count(name):
    return len(glob.glob(os.path.join(BLOCKS_DATASET_DIR, name, '*.jpg')))


class Saver(object):
    
    def __init__(self, name, count_widget):
        self.name = name
        self.count_widget = count_widget
        
    def save(self, x):
        image_path = os.path.join(BLOCKS_DATASET_DIR, self.name, str(uuid1()) + '.jpg')
        with open(image_path, 'wb') as f:
            f.write(image_widget.value)
        self.count_widget.value = get_count(self.name)
        
    
# create thumbnail widgets
class_widgets = []
button_widgets = []
count_widgets = []
for i, name in enumerate(class_names):
    # create directory for class
    try:
        os.makedirs(os.path.join(BLOCKS_DATASET_DIR, name))
    except:
        pass
    
    # get thumbnail widget
    thumbnail_widget = widgets.Image(format='jpeg', width=80, height=80)
    thumbnail_path = os.path.join(THUMBNAIL_DIR, name + '.jpg')
    with open(thumbnail_path, 'rb') as f:
        thumbnail_widget.value = f.read()
    
    # get count widget
    count_layout = widgets.Layout(width='80px')
    count_widget = widgets.IntText(layout=count_layout)
    
    # get button
    button_layout = widgets.Layout(width='80px', height='50px')
    button_widget = widgets.Button(description=name, layout=button_layout)
    
    saver = Saver(name, count_widget)
    
    button_widget.on_click(saver.save)
    
    class_widget = widgets.VBox([
        thumbnail_widget,
        count_widget,
        button_widget,
    ])
    
    class_widgets.append(class_widget)

display(widgets.HBox(class_widgets))

In [65]:
!zip -r -q lego_city_{BLOCKS_DATASET_DIR}_{uuid1()}.zip {BLOCKS_DATASET_DIR}